# Quantitative Risk Engine - Interactive Mode

This notebook interactively calls the `run_risk.py` script functions to perform risk analysis.

---

## Setup: Import the QuantLab Library

In [ ]:
# Add src to path and import required modules
import sys
from pathlib import Path

# Add the src directory to Python path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / "src"))

# Import quantlab modules
import matplotlib.pyplot as plt
import numpy as np

# Standard libraries
import pandas as pd
import seaborn as sns

from quantlab.config import settings
from quantlab.data import clean_prices, load_prices
from quantlab.risk import es_all, garch_fit, garch_forecast, garch_var, stress_run_all, var_all

# Setup
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

print("QuantLab Risk Engine loaded successfully!")
print(f"Project root: {project_root}")

---

## Step 1: Configure Portfolio Parameters

Define your portfolio composition:

In [ ]:
# ============================================================
# INTERACTIVE CONFIGURATION
# Modify the portfolio weights as needed
# ============================================================

# Portfolio Value (in dollars)
PORTFOLIO_VALUE = 1_000_000

# Date Range
START_DATE = '2018-01-01'
END_DATE = '2024-12-01'

# Confidence Level for VaR
CONFIDENCE = 0.95

# Portfolio Weights (must sum to 1.0)
PORTFOLIO = {
    # Technology (20%)
    'AAPL': 0.08, 'MSFT': 0.08, 'NVDA': 0.04,
    # Healthcare (15%)
    'JNJ': 0.06, 'UNH': 0.05, 'PFE': 0.04,
    # Financials (20%)
    'JPM': 0.08, 'BAC': 0.06, 'GS': 0.06,
    # Consumer (20%)
    'WMT': 0.05, 'PG': 0.05, 'KO': 0.05, 'MCD': 0.05,
    # Energy & Industrials (15%)
    'XOM': 0.05, 'CVX': 0.05, 'CAT': 0.05,
    # Other (10%)
    'V': 0.05, 'DIS': 0.05,
}

# Verify weights sum to 1
total_weight = sum(PORTFOLIO.values())
print("Configuration:")
print(f"  Portfolio Value: ${PORTFOLIO_VALUE:,.0f}")
print(f"  Period: {START_DATE} to {END_DATE}")
print(f"  Confidence Level: {CONFIDENCE*100:.0f}%")
print(f"  Stocks: {len(PORTFOLIO)}")
print(f"  Total Weight: {total_weight:.2%}")

if abs(total_weight - 1.0) > 0.01:
    print("\n WARNING: Weights don't sum to 100%. Normalizing...")

In [ ]:
# Display portfolio allocation
portfolio_df = pd.DataFrame([
    {'Ticker': k, 'Weight': v, 'Value': v * PORTFOLIO_VALUE}
    for k, v in PORTFOLIO.items()
]).sort_values('Weight', ascending=False)

print("Portfolio Allocation:")
display(portfolio_df)

# Pie chart
fig, ax = plt.subplots(figsize=(10, 8))
colors = plt.cm.tab20(np.linspace(0, 1, len(PORTFOLIO)))
ax.pie(portfolio_df['Weight'], labels=portfolio_df['Ticker'], autopct='%1.1f%%',
       colors=colors, startangle=90)
ax.set_title('Portfolio Allocation', fontweight='bold', fontsize=14)
plt.tight_layout()
plt.show()

---

## Step 2: Load Market Data

In [ ]:
# Load data
tickers = list(PORTFOLIO.keys())
weights = np.array(list(PORTFOLIO.values()))

print(f"Loading data for {len(tickers)} stocks...")
prices = load_prices(tickers, START_DATE, END_DATE)
prices = clean_prices(prices)
returns = prices.pct_change().dropna()

# Align weights with available data
available = [t for t in tickers if t in returns.columns]
weights_available = np.array([PORTFOLIO[t] for t in available])
weights_available = weights_available / weights_available.sum()  # Renormalize

returns = returns[available]

# Calculate portfolio returns
portfolio_returns = (returns * weights_available).sum(axis=1)

print("\nData loaded:")
print(f"  Trading days: {len(returns)}")
print(f"  Stocks with data: {len(available)}/{len(tickers)}")

In [ ]:
# Visualize portfolio returns distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
ax1 = axes[0]
ax1.hist(portfolio_returns * 100, bins=50, density=True, alpha=0.7, color='steelblue', edgecolor='white')
ax1.axvline(x=0, color='black', linestyle='--', linewidth=1)
ax1.axvline(x=portfolio_returns.mean() * 100, color='green', linestyle='-', linewidth=2, label=f'Mean: {portfolio_returns.mean()*100:.3f}%')
ax1.set_title('Portfolio Daily Returns Distribution', fontweight='bold')
ax1.set_xlabel('Daily Return (%)')
ax1.set_ylabel('Density')
ax1.legend()

# Cumulative returns
ax2 = axes[1]
cumulative = (1 + portfolio_returns).cumprod()
ax2.plot(cumulative.index, cumulative.values, color='blue', linewidth=1.5)
ax2.fill_between(cumulative.index, 1, cumulative.values, alpha=0.3)
ax2.axhline(y=1, color='black', linestyle='--', alpha=0.5)
ax2.set_title('Portfolio Cumulative Return', fontweight='bold')
ax2.set_ylabel('Cumulative Return')

plt.tight_layout()
plt.show()

print("\nPortfolio Statistics:")
print(f"  Mean Daily Return: {portfolio_returns.mean()*100:.4f}%")
print(f"  Daily Volatility: {portfolio_returns.std()*100:.4f}%")
print(f"  Annualized Return: {portfolio_returns.mean()*252*100:.2f}%")
print(f"  Annualized Volatility: {portfolio_returns.std()*np.sqrt(252)*100:.2f}%")

---

## Step 3: Value at Risk (VaR) Analysis

Calculate VaR using multiple methods:

In [ ]:
# Calculate VaR using quantlab functions
print("Calculating Value at Risk (VaR)...")
print("="*60)

var_results = var_all(returns, weights=weights_available)

print("\nVaR Results (as % of portfolio):")
display(var_results * 100)

# Convert to dollar amounts
print(f"\nVaR in Dollar Terms (Portfolio: ${PORTFOLIO_VALUE:,.0f}):")
print("-"*60)
print(f"{'Method':<20} {'95% VaR':>20} {'99% VaR':>20}")
print("-"*60)

for method in var_results.columns:
    var_95 = var_results.loc['95%', method] * PORTFOLIO_VALUE
    var_99 = var_results.loc['99%', method] * PORTFOLIO_VALUE
    print(f"{method.title():<20} ${var_95:>19,.0f} ${var_99:>19,.0f}")

---

## Step 4: Expected Shortfall (CVaR) Analysis

In [ ]:
# Calculate Expected Shortfall
print("Calculating Expected Shortfall (ES/CVaR)...")
print("="*60)

es_results = es_all(returns, weights=weights_available)

print("\nExpected Shortfall Results (as % of portfolio):")
display(es_results * 100)

# ES vs VaR comparison
print("\nES vs VaR Comparison:")
print("-"*60)
for method in ['historical', 'parametric']:
    var_95 = var_results.loc['95%', method]
    es_95 = es_results.loc['95%', method]
    ratio = es_95 / var_95 if var_95 > 0 else 0
    print(f"{method.title()}: ES/VaR ratio = {ratio:.2f}x (ES is {ratio:.2f}x worse than VaR in tail events)")

In [ ]:
# Visualize VaR and ES
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# VaR comparison
ax1 = axes[0]
methods = var_results.columns
x = np.arange(len(methods))
width = 0.35

var_95_vals = [var_results.loc['95%', m] * 100 for m in methods]
var_99_vals = [var_results.loc['99%', m] * 100 for m in methods]

ax1.bar(x - width/2, var_95_vals, width, label='95% VaR', color='orange', alpha=0.7)
ax1.bar(x + width/2, var_99_vals, width, label='99% VaR', color='red', alpha=0.7)
ax1.set_ylabel('VaR (%)')
ax1.set_title('Value at Risk by Method', fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels([m.title() for m in methods])
ax1.legend()
ax1.grid(True, alpha=0.3)

# ES comparison
ax2 = axes[1]
es_95_vals = [es_results.loc['95%', m] * 100 for m in es_results.columns]
es_99_vals = [es_results.loc['99%', m] * 100 for m in es_results.columns]

ax2.bar(x - width/2, es_95_vals, width, label='95% ES', color='orange', alpha=0.7)
ax2.bar(x + width/2, es_99_vals, width, label='99% ES', color='red', alpha=0.7)
ax2.set_ylabel('ES (%)')
ax2.set_title('Expected Shortfall by Method', fontweight='bold')
ax2.set_xticks(x)
ax2.set_xticklabels([m.title() for m in es_results.columns])
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(str(project_root / 'reports' / 'figures' / 'var_es_comparison.png'), dpi=150, bbox_inches='tight')
plt.show()

---

## Step 5: GARCH Volatility Modeling

In [ ]:
# Fit GARCH model
print("Fitting GARCH(1,1) model...")
print("="*60)

garch_result, garch_params = garch_fit(portfolio_returns)

print("\nGARCH(1,1) Model Parameters:")
print(f"  Omega (constant): {garch_params.get('omega', 0):.6f}")
print(f"  Alpha (shock impact): {garch_params['alpha']:.4f}")
print(f"  Beta (persistence): {garch_params['beta']:.4f}")
print(f"  Persistence (alpha + beta): {garch_params['persistence']:.4f}")

# Volatility forecast
forecast = garch_forecast(garch_result, horizon=5)
print("\nVolatility Forecast (next 5 days):")
for i, vol in enumerate(forecast['volatility'].values):
    print(f"  Day {i+1}: {vol*100:.2f}%")

In [ ]:
# GARCH-based VaR
next_day_vol = forecast['volatility'].iloc[0]
garch_var_95 = garch_var(next_day_vol, 0.95)
garch_var_99 = garch_var(next_day_vol, 0.99)

print("\nGARCH-based VaR (using forecasted volatility):")
print(f"  95% VaR: {garch_var_95*100:.2f}% (${garch_var_95*PORTFOLIO_VALUE:,.0f})")
print(f"  99% VaR: {garch_var_99*100:.2f}% (${garch_var_99*PORTFOLIO_VALUE:,.0f})")

In [ ]:
# Visualize GARCH conditional volatility
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Plot 1: Portfolio returns with volatility bands
ax1 = axes[0]
cond_vol = garch_result.conditional_volatility / 100  # Convert to decimal
ax1.plot(portfolio_returns.index, portfolio_returns * 100, alpha=0.5, linewidth=0.5, color='blue', label='Returns')
ax1.plot(cond_vol.index, cond_vol * 100 * 2, color='red', linewidth=1, label='+2 Std Dev')
ax1.plot(cond_vol.index, -cond_vol * 100 * 2, color='red', linewidth=1, label='-2 Std Dev')
ax1.fill_between(cond_vol.index, cond_vol * 100 * 2, -cond_vol * 100 * 2, alpha=0.1, color='red')
ax1.set_title('Portfolio Returns with GARCH Volatility Bands', fontweight='bold')
ax1.set_ylabel('Return (%)')
ax1.legend(loc='upper right')

# Plot 2: Conditional volatility over time
ax2 = axes[1]
ax2.plot(cond_vol.index, cond_vol * 100 * np.sqrt(252), color='orange', linewidth=1)
ax2.fill_between(cond_vol.index, 0, cond_vol * 100 * np.sqrt(252), alpha=0.3, color='orange')
ax2.set_title('GARCH Conditional Volatility (Annualized)', fontweight='bold')
ax2.set_ylabel('Volatility (%)')
ax2.set_xlabel('Date')

plt.tight_layout()
plt.savefig(str(project_root / 'reports' / 'figures' / 'garch_volatility.png'), dpi=150, bbox_inches='tight')
plt.show()

---

## Step 6: Stress Testing

In [ ]:
# Run stress tests
print("Running stress tests...")
print("="*60)

stress_results = stress_run_all(PORTFOLIO, PORTFOLIO_VALUE)

print(f"\nStress Test Results (Portfolio: ${PORTFOLIO_VALUE:,.0f}):")
print("-"*70)
print(f"{'Scenario':<35} {'Return':>12} {'Dollar Loss':>18}")
print("-"*70)

for name, result in sorted(stress_results.items(), key=lambda x: x[1].dollar_loss, reverse=True):
    ret = result.portfolio_return * 100
    loss = result.dollar_loss
    print(f"{result.scenario_name:<35} {ret:>11.1f}% ${loss:>17,.0f}")

In [ ]:
# Visualize stress test results
fig, ax = plt.subplots(figsize=(12, 6))

scenarios = [stress_results[k].scenario_name for k in stress_results]
losses = [stress_results[k].dollar_loss for k in stress_results]

# Sort by loss
sorted_data = sorted(zip(scenarios, losses), key=lambda x: x[1], reverse=True)
scenarios, losses = zip(*sorted_data)

colors = ['darkred' if l > PORTFOLIO_VALUE * 0.1 else 'red' if l > PORTFOLIO_VALUE * 0.05 else 'orange' for l in losses]

bars = ax.barh(range(len(scenarios)), losses, color=colors, alpha=0.7)
ax.set_yticks(range(len(scenarios)))
ax.set_yticklabels(scenarios)
ax.set_xlabel('Dollar Loss')
ax.set_title('Stress Test Results - Potential Losses by Scenario', fontweight='bold')

# Add VaR reference line
var_99_hist = var_results.loc['99%', 'historical'] * PORTFOLIO_VALUE
ax.axvline(x=var_99_hist, color='blue', linestyle='--', linewidth=2, label=f'99% Historical VaR: ${var_99_hist:,.0f}')
ax.legend(loc='lower right')

# Format x-axis
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))

plt.tight_layout()
plt.savefig(str(project_root / 'reports' / 'figures' / 'stress_test_results.png'), dpi=150, bbox_inches='tight')
plt.show()

---

## Step 7: Risk Summary Report

In [ ]:
# Generate comprehensive risk summary
print("\n" + "="*70)
print("                    RISK ANALYSIS SUMMARY")
print("="*70)
print(f"\nPortfolio Value: ${PORTFOLIO_VALUE:,.0f}")
print(f"Period: {START_DATE} to {END_DATE}")

# VaR Summary
print("\n" + "-"*70)
print("VALUE AT RISK (VaR)")
print("-"*70)
hist_var_95 = var_results.loc['95%', 'historical']
hist_var_99 = var_results.loc['99%', 'historical']
print(f"  95% 1-day VaR: {hist_var_95*100:.2f}% (${hist_var_95*PORTFOLIO_VALUE:,.0f})")
print(f"  99% 1-day VaR: {hist_var_99*100:.2f}% (${hist_var_99*PORTFOLIO_VALUE:,.0f})")

# ES Summary
print("\n" + "-"*70)
print("EXPECTED SHORTFALL (ES/CVaR)")
print("-"*70)
hist_es_95 = es_results.loc['95%', 'historical']
hist_es_99 = es_results.loc['99%', 'historical']
print(f"  95% 1-day ES: {hist_es_95*100:.2f}% (${hist_es_95*PORTFOLIO_VALUE:,.0f})")
print(f"  99% 1-day ES: {hist_es_99*100:.2f}% (${hist_es_99*PORTFOLIO_VALUE:,.0f})")
print(f"  ES/VaR Ratio (95%): {hist_es_95/hist_var_95:.2f}x")

# GARCH Summary
print("\n" + "-"*70)
print("GARCH VOLATILITY MODEL")
print("-"*70)
print("  Model: GARCH(1,1)")
print(f"  Persistence: {garch_params['persistence']:.4f}")
print(f"  Next-day Vol Forecast: {next_day_vol*100:.2f}%")
print(f"  GARCH 95% VaR: {garch_var_95*100:.2f}% (${garch_var_95*PORTFOLIO_VALUE:,.0f})")

# Stress Test Summary
print("\n" + "-"*70)
print("STRESS TESTING")
print("-"*70)
worst_stress = max(stress_results.values(), key=lambda x: x.dollar_loss)
print(f"  Worst Case Scenario: {worst_stress.scenario_name}")
print(f"  Worst Case Loss: ${worst_stress.dollar_loss:,.0f} ({worst_stress.portfolio_return*100:.1f}%)")
print(f"  Stress/VaR Ratio: {worst_stress.dollar_loss/(hist_var_99*PORTFOLIO_VALUE):.1f}x")

print("\n" + "="*70)

In [ ]:
# Save results to CSV
settings.ensure_dirs()
output_path = settings.processed_data_dir / "risk_results_interactive.csv"

summary = pd.DataFrame({
    'metric': [
        'VaR_95_hist', 'VaR_99_hist',
        'ES_95_hist', 'ES_99_hist',
        'GARCH_vol_forecast', 'GARCH_VaR_95',
        'Worst_stress_loss'
    ],
    'value': [
        hist_var_95, hist_var_99,
        hist_es_95, hist_es_99,
        next_day_vol, garch_var_95,
        worst_stress.dollar_loss
    ],
    'dollar_value': [
        hist_var_95 * PORTFOLIO_VALUE, hist_var_99 * PORTFOLIO_VALUE,
        hist_es_95 * PORTFOLIO_VALUE, hist_es_99 * PORTFOLIO_VALUE,
        None, garch_var_95 * PORTFOLIO_VALUE,
        worst_stress.dollar_loss
    ]
})

summary.to_csv(output_path, index=False)
print(f"Results saved to {output_path}")

display(summary)

---

## Notebook Complete

You can now:
1. Modify portfolio weights in Step 1
2. Change the date range or confidence levels
3. Add custom stress scenarios
4. Export results for reporting